<a href="https://colab.research.google.com/github/varaha-Maithreya/Person-Reidentification/blob/main/videoTrack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/varaha-Maithreya/Person-Reidentification.git

Cloning into 'Person-Reidentification'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 133 (delta 13), reused 114 (delta 6), pack-reused 5
Receiving objects: 100% (133/133), 353.01 KiB | 20.76 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [2]:
pip install timm scipy einops yacs opencv-python tensorboard pandas pytorchvideo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 548 kB 16.8 MB/s 
     |████████████████████████████████| 132 kB 47.9 MB/s 
     |████████████████████████████████| 163 kB 51.3 MB/s 
     |████████████████████████████████| 50 kB 5.3 MB/s 
     |████████████████████████████████| 28.2 MB 2.2 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188714 sha256=662488aed7c41986137a0bcba5ff1ac6ff07af60ea006fa429aef9634f809d5a
  Stored in directory: /root/.cache/pip/wheels/e8/51/05/053b29bac2400cbbae2fb7cfc41afd280d627bca7c9363ca80
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=04cbdfcf385d62ce2ef4b52a243963310eadc1ea992f5ca8f261b01d735ba299
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel f

In [3]:
!ls 

!pwd

drive  Person-Reidentification	sample_data
/content


In [4]:
cd Person-Reidentification/


/content/Person-Reidentification


In [5]:
import os
import torch, cv2
import numpy as np
from config import cfg
import argparse
from datasets import make_dataloader
from model import make_model
from processor import do_inference
from utils.logger import setup_logger
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
from model.make_model import __num_of_layers
from model.make_model import build_transformer_local
from model.backbones.vit_pytorch import vit_base_patch16_224_PiT, vit_small_patch16_224_PiT, deit_small_patch16_224_PiT

In [6]:
cfg.merge_from_file("/content/Person-Reidentification/configs/iLIDS-VID/pit.yml")

cfg.merge_from_list(['TEST.VIS', "True"])
cfg.freeze()

In [7]:
output_dir = cfg.OUTPUT_DIR
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [8]:
torch.cuda.is_available()
torch.cuda.device_count()

1

In [9]:
os.environ['CUDA_VISIBLE_DEVICES'] = cfg.MODEL.DEVICE_ID

In [10]:
os.environ['CUDA_VISIBLE_DEVICES'] 

'6'

In [ ]:
epochs = cfg.SOLVER.MAX_EPOCHS
eval_period = cfg.SOLVER.EVAL_PERIOD
OUTPUT_DIR = cfg.TEST.WEIGHT


In [ ]:
val_transforms = T.Compose([
   T.Resize(cfg.INPUT.SIZE_TEST, interpolation=3),
   T.ToTensor(),
   T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


In [ ]:
__factory_T_type = {
    'vit_base_patch16_224_PiT': vit_base_patch16_224_PiT,
    'deit_base_patch16_224_PiT': vit_base_patch16_224_PiT,
    'vit_small_patch16_224_PiT': vit_small_patch16_224_PiT,
    'deit_small_patch16_224_PiT': deit_small_patch16_224_PiT
}

__num_of_layers = {
    '1x210' : 1,    # global
    '2x105' : 2,    # horizontal
    '3x70'  : 3,
    '5x42'  : 5,
    '6x35'  : 6,
    '7x10'  : 7,
    '105x2' : 2,    # vertical
    '70x3'  : 3,
    '42x5'  : 5,
    '35x6'  : 6,
    '10x7'  : 7,
    '6p'    : 6,    # patch
    '14p'   : 14,
    '15p'   : 15,
    'NULL'  : 0,
}

In [ ]:
train_loader, val_loader, num_query, num_classes, camera_num, view_num = make_dataloader(cfg)



This dataset has been downloaded.
Creating splits
Totally 10 splits are created, following Wang et al. ECCV'14
Split file is saved to ../../data/i-LIDS-VID/splits.json
Splits created
# train identites: 150, # test identites 150
=> iLIDS-VID loaded
Dataset statistics:
  ------------------------------
  subset   | # ids | # tracklets
  ------------------------------
  train    |   150 |      300
  query    |   150 |      150
  gallery  |   150 |      150
  ------------------------------
  total    |   300 |      600
  number of images per tracklet: 22 ~ 192, average 70.8
  ------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
!pwd
num_trails = 10

model = make_model(cfg, num_class=num_classes, camera_num=camera_num, view_num = view_num)

param_dict = torch.load('/content/jx_vit_base_p16_224-80ecf9dd.pth')
model.load_state_dict(param_dict, strict=False)




model.to("cuda")
model.eval()



/content/Person-Reidentification
using Transformer_type: vit_base_patch16_224_PiT as a backbone
using stride: [12, 12], and patch number is num_y21 * num_x10
camera number is : 2
using SIE_Lambda is : 3.0
using drop_out rate is : 0.0
using attn_drop_out rate is : 0.0
using drop_path rate is : 0.1
Resized position embedding from size:torch.Size([1, 197, 768]) to size: torch.Size([1, 211, 768]) with height:21 width: 10
Loading pretrained ImageNet model......from /content/jx_vit_base_p16_224-80ecf9dd.pth
using shuffle_groups size:2
using shift_num size:5
using divide_length size:5
===========building transformer with JPM module ===========


build_transformer_local(
  (base): PiT(
    (patch_embed): PatchEmbed_overlap(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(12, 12))
    )
    (temporal_pos_drop): Dropout(p=0.0, inplace=False)
    (spatial_blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate=none)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Bl

In [35]:
from PIL import Image
directory = '/content/img.png'
img = Image.open(directory).convert('RGB')
val_transforms(img)
img = torch.stack([val_transforms(img)], 0).unsqueeze(0).to("cuda")
attns = model(img, cam_label=[0])





In [39]:


import torch
import json
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
from typing import Dict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 27.2 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
     |████████████████████████████████| 28.2 MB 1.5 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188714 sha256=77ae515109b8b460b62b9cf8dcc95ebeeb255ea9b4bc07d08ba09aca115bd577
  Stored in directory: /root/.cache/pip/wheels/e8/51/05/053b29bac2400cbbae2fb7cfc41afd280d627bca7c9363ca80
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=100ef03011176e72608a36bbd54c9b1bd94e72fcc14966ff06e180f62fcabd39
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31549 sha256=b796001f61a7a0912f424fef2418231cc834dc8c434dda7

In [2]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
alpha = 4

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate)/frames_per_second

NameError: ignored

In [1]:
video_path = "/content/drive/MyDrive/cam4.mp4"

# Select the duration of the clip to load by specifying the start and end duration
# The start_sec should correspond to where the action occurs in the video
start_sec = 0
end_sec = start_sec + 150

# Initialize an EncodedVideo helper class
video = EncodedVideo.from_path(video_path)

# Load the desired clip
video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

# Apply a transform to normalize the video input
video_data = transform(video_data)

# Move the inputs to the desired device
inputs = video_data["video"]
inputs = [i.to(device)[None, ...] for i in inputs]

score, feat, diversity = model(img, inputs , cam_label='0', view_label='view_0' )

score

NameError: ignored